# 📚 Poisson Surface Reconstruction Notebook (20 Points)
## 🔍 From Point Clouds to Smooth Surfaces

In [1]:
# 🛠️ Environment Setup
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse.linalg import LinearOperator, cg
from scipy.interpolate import RegularGridInterpolator
from skimage.measure import marching_cubes
from IPython.display import Markdown, display

# 🎨 Visualization Config
plt.rcParams['figure.figsize'] = [10, 6]

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## 📘 Algorithm Implementation (20 Points)
**Key Idea:** Create explicit surfaces from oriented point clouds by solving the Poisson equation

**Main Steps:**
1. 🧭 Vector Field Construction
2. 🌀 Divergence Calculation
3. ⚡ Poisson Equation Solution
4. ✂️ Iso-surface Extraction

**Mathematical Foundation:**
$$
\nabla²\chi = \nabla\cdot V \rightarrow \text{Solve for } \chi \rightarrow \text{Extract } \{x | \chi(x) = 0\}
$$

Complete the implementation by solving
- **Exercise 1: Compute the divergence operator on the vector field V via finite differences**
- **Exercise 2: Solve for χ by defining Laplacian as a Linear Operator A and divergence as a right hand side of Aχ=b**
Solve the optimization problem with Conjugate Gradient descent with A of shape $$(nx*ny*nz, nx*ny*nz)$$
For ease of use, we have already provided the implementation of laplacian for you

In [5]:
import open3d as o3d
import numpy as np
from scipy.sparse.linalg import LinearOperator, cg
from scipy.interpolate import RegularGridInterpolator
from skimage.measure import marching_cubes

# 🌀 PoissonReconstructor Class Implementation
class PoissonReconstructor:
    def __init__(self, grid_resolution=64, kernel_sigma=0.1):
        """
        Poisson Surface Reconstruction Algorithm

        Parameters:
        grid_resolution (int): Number of voxels along each axis (controls detail)
        kernel_sigma (float): Gaussian spread radius for normal distribution
        """
        # Grid configuration
        self.grid_resolution = grid_resolution  # Voxel grid size (N x N x N)
        self.kernel_sigma = kernel_sigma        # Gaussian kernel standard deviation

        # Spatial properties
        self.bbox = None       # Bounding box [[min_x,y,z], [max_x,y,z]] with padding
        self.voxel_size = None # Physical size of each voxel (dx, dy, dz)
        self.grid = None       # Tuple of (x_coords, y_coords, z_coords) arrays

        # Vector field components
        self.Vx = self.Vy = self.Vz = None  # 3D grid of vector field values

        # Intermediate calculations
        self.divV = None  # Divergence of vector field (∇·V)
        self.chi = None   # Solution to Poisson equation (potential field)
        self.f = None     # Shifted potential field for iso-surface extraction

    def fit(self, points, normals):
        """
        Main reconstruction pipeline

        Parameters:
        points : (N,3) array of 3D point coordinates
        normals : (N,3) array of corresponding surface normals
        """
        # ================== 1. Grid Setup ==================
        # Compute bounding box with 3σ padding for kernel operations
        self.bbox = np.array([np.min(points, axis=0), np.max(points, axis=0)])
        padding = 3 * self.kernel_sigma
        self.bbox[0] -= padding  # Expand min bounds
        self.bbox[1] += padding  # Expand max bounds

        # Calculate voxel size based on bounding box dimensions
        bbox_size = self.bbox[1] - self.bbox[0]
        self.voxel_size = bbox_size / self.grid_resolution
        nx, ny, nz = [self.grid_resolution] * 3  # Grid dimensions
        h = self.voxel_size[0]  # Assume uniform voxel size (dx = dy = dz)

        # Create linearly spaced grid coordinates
        x = np.linspace(self.bbox[0][0], self.bbox[1][0], nx)
        y = np.linspace(self.bbox[0][1], self.bbox[1][1], ny)
        z = np.linspace(self.bbox[0][2], self.bbox[1][2], nz)
        self.grid = (x, y, z)

        # =========== 2. Vector Field Construction ===========
        # Initialize vector field components to zero
        self.Vx = np.zeros((nx, ny, nz))
        self.Vy = np.zeros((nx, ny, nz))
        self.Vz = np.zeros((nx, ny, nz))

        # Spread normals to grid using Gaussian kernel
        kernel_radius = 3 * self.kernel_sigma  # 3σ contains ~99.7% of distribution
        for p, n in zip(points, normals):
            # Find grid neighborhood around current point
            dx = int(np.ceil(kernel_radius / h))  # Grid steps within kernel radius
            # Find center indices using binary search
            i_center = np.clip(np.searchsorted(x, p[0], side='right') - 1, 0, nx-1)
            j_center = np.clip(np.searchsorted(y, p[1], side='right') - 1, 0, ny-1)
            k_center = np.clip(np.searchsorted(z, p[2], side='right') - 1, 0, nz-1)

            # Define neighborhood bounds
            imin, imax = max(0, i_center - dx), min(nx, i_center + dx + 1)
            jmin, jmax = max(0, j_center - dx), min(ny, j_center + dx + 1)
            kmin, kmax = max(0, k_center - dx), min(nz, k_center + dx + 1)

            # Accumulate weighted normals in neighborhood
            for i in range(imin, imax):
                for j in range(jmin, jmax):
                    for k in range(kmin, kmax):
                        g_pos = (x[i], y[j], z[k])  # Grid point position
                        d = np.linalg.norm(g_pos - p)
                        if d > kernel_radius:
                            continue
                        # Gaussian weighting: exp(-d²/(2σ²))
                        # Your Code Here
                        weight = np.exp(-d**2/(2*self.kernel_sigma**2))
                        self.Vx[i, j, k] += weight * n[0]
                        self.Vy[i, j, k] += weight * n[1]
                        self.Vz[i, j, k] += weight * n[2]

        # ============ 3. Divergence Calculation =============
        # Your implementation should start here ...
        self.divV = np.zeros((nx, ny, nz))
        for i in range(nx):
            for j in range(ny):
                for k in range(nz):
                    # Finite differences with forward, backward, and left finite differences
                    # Your Code Here (complete missing parts of this block of code. Missing parts are signed with an elipses)
                    dVx = 0.0 # X-component
                    if i == 0:
                        dVx = (self.Vx[i+1, j, k] - self.Vx[i, j, k]) / (x[i+1] - x[i])
                    elif i == nx-1:
                        dVx = (self.Vx[i, j, k] - self.Vx[i-1, j, k]) / (x[i] - x[i-1])
                    else:
                        dVx = (self.Vx[i+1, j, k] - self.Vx[i-1, j, k]) / (x[i+1] - x[i-1])

                    dVy = 0.0 # Y-component
                    if j == 0: # Forward difference at left boundary
                        dVy = (self.Vy[i, j+1, k] - self.Vy[i, j, k]) / (y[j+1] - y[j])
                    elif j == ny-1: # Backward difference at right boundary
                        dVy = (self.Vy[i, j, k] - self.Vy[i, j-1, k]) / (y[j] - y[j-1])
                    else:
                        dVy = (self.Vy[i, j+1, k] - self.Vy[i, j-1, k]) / (y[j+1] - y[j-1])

                    dVz = 0.0 # Z-component
                    if k == 0: # Forward difference at left boundary
                        dVz = (self.Vz[i, j, k+1] - self.Vz[i, j, k]) / (z[k+1] - z[k])
                    elif k == nz-1: # Backward difference at right boundary
                        dVz = (self.Vz[i, j, k] - self.Vz[i, j, k-1]) / (z[k] - z[k-1])
                    else: # Central difference
                        dVz = (self.Vz[i, j, k+1] - self.Vz[i, j, k-1]) / (z[k+1] - z[k-1])

                    self.divV[i,j,k] = dVx + dVy + dVz  # ∇·V

        # ============ 4. Poisson Equation Solve =============
        def laplacian_vec(chi_flat):
            """Implements discrete Laplacian operator with Neumann BC"""
            chi = chi_flat.reshape((nx, ny, nz)).astype(np.float64)
            lap = np.zeros_like(chi)

            # x-direction Derivatives
            if nx > 1:
              lap[1:-1, :, :] += (chi[2:, :, :] - 2*chi[1:-1, :, :] + chi[:-2, :, :]) / h**2
              # Neumann BC: derivative=0 → forward/backward differences at boundaries
              lap[0, :, :] += (chi[1, :, :] - chi[0, :, :]) / h**2
              lap[-1, :, :] += (chi[-2, :, :] - chi[-1, :, :]) / h**2

            # y-direction Derivatives
            if ny > 1:
              # Interior points: central difference
              lap[:, 1:-1, :] += (chi[:, 2:, :] - 2*chi[:, 1:-1, :] + chi[:, :-2, :]) / h**2
              # Neumann BC: derivative=0 → forward/backward differences at boundaries
              lap[:, 0, :] += (chi[:, 1, :] - chi[:, 0, :]) / h**2
              lap[:, -1, :] += (chi[:, -2, :] - chi[:, -1, :]) / h**2

            # z-direction Derivatives
            if nz > 1:
              # Interior points: central difference
              lap[:, :, 1:-1] += (chi[:, :, 2:] - 2*chi[:, :, 1:-1] + chi[:, :, :-2]) / h**2
              # Neumann BC: derivative=0 → forward/backward differences at boundaries
              lap[:, :, 0] += (chi[:, :, 1] - chi[:, :, 0]) / h**2
              lap[:, :, -1] += (chi[:, :, -2] - chi[:, :, -1]) / h**2

            return lap.ravel()

        # Prepare right-hand side (divV) with zero mean for Neumann BC compatibility
        b = self.divV.ravel()
        b -= np.mean(b)

        # Solve ∇²χ = ∇·V using Conjugate Gradient method
        # Your implementation should start here ...
        # Solve for χ by defining Laplacian as a Linear Operator A and divergence as a right hand side of Aχ=b**
        N = nx * ny * nz
        A = LinearOperator(shape=(N,N), matvec=laplacian_vec)
        chi_flat, info = cg(A, b)
        if info != 0:
            raise RuntimeError("CG solver failed to converge")
        self.chi = chi_flat.reshape((nx, ny, nz))

        # ========= 5. Iso-Surface Adjustment =========
        # Find iso-value as average χ at input points
        interpolator = RegularGridInterpolator((x, y, z), self.chi)
        chi_at_points = interpolator(points)
        iso_value = np.mean(chi_at_points)
        self.f = self.chi - iso_value  # Shift to get f=0 surface

    def extract_mesh(self):
        """Extract mesh using Marching Cubes algorithm"""
        if self.f is None:
            raise ValueError("Must call fit() first")
        # Run Marching Cubes at level 0 (our iso-surface)
        vertices, faces, _, _ = marching_cubes(
            self.f,
            level=0,
            spacing=self.voxel_size
        )
        # Transform from grid coordinates to world coordinates
        vertices += self.bbox[0]
        return vertices, faces

## 🦌 Dataset Preparation
Loading and visualizing sample point clouds from Open3D's built-in datasets:

In [6]:
# Load the Sphere mesh and create oriented point cloud
gt_mesh = o3d.geometry.TriangleMesh.create_sphere()
gt_mesh.compute_vertex_normals()

# Generate oriented point cloud
pcd = gt_mesh.sample_points_poisson_disk(5000)
pcd.normals = o3d.utility.Vector3dVector(np.zeros((1, 3)))  # Invalidate normals
pcd.estimate_normals()
pcd.orient_normals_consistent_tangent_plane(100)

# Visualize
o3d.visualization.draw_plotly([pcd])

## ⚙️ Reconstruction Pipeline
### Key Parameters:
- `grid_resolution`: 64
- `kernel_sigma`: 0.1

### Processing Steps:

In [ ]:
# Convert to numpy arrays
points = np.asarray(pcd.points)
normals = np.asarray(pcd.normals)

# Initialize and run reconstructor
pr = PoissonReconstructor(grid_resolution=64, kernel_sigma=0.1)
pr.fit(points, normals)
vertices, faces = pr.extract_mesh()

# Create and visualize mesh
recon_mesh = o3d.geometry.TriangleMesh()
recon_mesh.vertices = o3d.utility.Vector3dVector(vertices)
recon_mesh.triangles = o3d.utility.Vector3iVector(faces)
recon_mesh.compute_vertex_normals()

# Side-by-side comparison
pcd.paint_uniform_color([0.7, 0.3, 0.3])
recon_mesh.paint_uniform_color([0.3, 0.7, 0.3])
o3d.visualization.draw_plotly([pcd, recon_mesh])

**Expected Result**:

![Poisson Recon](./../data/poisson_recon.png)

## 🎓 Key Limitations of Poisson Surface Reconstruction
   - Computational complexity
   - Memory intensive
   - Requires normals